<a href="https://colab.research.google.com/github/mosfet-sudo/ML-Project-G75/blob/main/90049_A2_G75_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

note: the github must be public in order for us to view 'live updates' of any changes made to this notebook

# Simple Data Processing
(Done by Elijah)
{insert brief outline of what the simple pre-processing involved}

## Metro-PT3

## NASA

In [ ]:

import pandas as pd
import numpy as np

#Method for parsing text files into dataframe, takes a string input and returns a dataframe object
def parseTxt2Dataframe(String):
    with open(String, 'r', encoding='utf-8') as file:
        lines = []
        for line in file:
            newline = line.strip()
            newline = newline.split()
            lines.append(newline)
        new_df = pd.DataFrame(lines, columns=['Unit Number', 'Time in cycles',
                                  'Operational Setting 1', 'Operational Setting 2','Operational Setting 3',
                                  'T2','T24','T30','T50','P2','P15','P30','Nf','Nc','epr','Ps30','phi','NRf','NRc',
                                  'BPR','farB','htBleed','Nf_dmd','PCNfr_dmd','W31','W32'])
        num_cols = new_df.shape[1]
        for i in range(num_cols):
            new_df.iloc[:,i] = pd.to_numeric(new_df.iloc[:,i])


    return new_df

#normalizes the data, and checks for duplicates, takes dataframe object as input and returns a dataframe
def normalize_and_checkdupe(df):
    checkdf = df.drop(columns=['Unit Number','Time in cycles'])
    duplicateddf = checkdf.duplicated()
    no_dupe_df = df.loc[duplicateddf==False]
    num_cols = no_dupe_df.shape[1]
    num_rows = no_dupe_df.shape[0]
    for i in range(2,num_cols):
        max_val = no_dupe_df.iloc[:,i].max()
        min_val = no_dupe_df.iloc[:,i].min()
        val_range = max_val - min_val
        for j in range(num_rows):
            if val_range != 0:
                no_dupe_df.iat[j, i] = (no_dupe_df.iat[j, i] - min_val) / val_range
            else:
                no_dupe_df.iat[j, i] = -1

    return no_dupe_df


# parsing each text file into a dataframe
df_test1 = parseTxt2Dataframe('C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/CMaps/test_FD001.txt')
df_test2 = parseTxt2Dataframe('C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/CMaps/test_FD002.txt')
df_test3 = parseTxt2Dataframe('C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/CMaps/test_FD003.txt')
df_test4 = parseTxt2Dataframe('C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/CMaps/test_FD004.txt')
df_train1 = parseTxt2Dataframe('C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/CMaps/train_FD001.txt')
df_train2 = parseTxt2Dataframe('C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/CMaps/train_FD002.txt')
df_train3 = parseTxt2Dataframe('C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/CMaps/train_FD003.txt')
df_train4 = parseTxt2Dataframe('C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/CMaps/train_FD004.txt')



#combining the data into one dataset

df_all = pd.concat([df_test1,df_test2,df_test3,df_test4,df_train1,df_train2,df_train3,df_train4],ignore_index=True)
print(df_all.shape)
print(df_test1.shape)

## normalizing and checking for duplicates when combined
df_all_processed = normalize_and_checkdupe(df_all)

## normalizing and checking for duplicates when uncombined
df_test1_processed = normalize_and_checkdupe(df_test1)
#df_test2_processed = normalize_and_checkdupe(df_test2)
#df_test3_processed = normalize_and_checkdupe(df_test3)
#df_test4_processed = normalize_and_checkdupe(df_test4)
#df_train1_processed = normalize_and_checkdupe(df_train1)
#df_train2_processed = normalize_and_checkdupe(df_train2)
#df_train3_processed = normalize_and_checkdupe(df_train3)
#df_train4_processed = normalize_and_checkdupe(df_train4)



## testing and debug
pd.set_option('display.max_columns', None)
print(df_all_processed.shape)
print(df_all_processed.head())
print(df_test1_processed.head())
print("newline")


print("newline")
print(df_all_processed.shape)

pd.reset_option('display.max_columns')
df_all_processed.to_csv('NASAprocessed.csv')

## PdM

In [ ]:
import pandas as pd
import numpy as np
import json
df = pd.read_csv("C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/ai_2020.csv")
#Duplicate checking

df = df.drop(columns = ["UDI"])
num_columns = df.shape[1]
num_rows = df.shape[0]
duplicateddf=df.duplicated()
no_duplicate_df = df.loc[duplicateddf==False]

print(no_duplicate_df.columns)
#Normalization

temp_indexes = [no_duplicate_df.columns.get_loc("Air temperature [K]")]
temp_indexes.append(no_duplicate_df.columns.get_loc("Process temperature [K]"))
temp_indexes.append(no_duplicate_df.columns.get_loc("Rotational speed [rpm]")) #TURN INTO SECONDS?
temp_indexes.append(no_duplicate_df.columns.get_loc("Torque [Nm]"))
temp_indexes.append(no_duplicate_df.columns.get_loc("Tool wear [min]"))     # TURN INTO SECONDS?
# BOTH RPM AND TOOLWEAR HAVE ISSUES WITH CERTAIN DTYPES BEING INT DUE TO WHOLE NUMBERS IN DATA
no_duplicate_df["Rotational speed [rpm]"] = no_duplicate_df["Rotational speed [rpm]"].astype(float)
no_duplicate_df["Tool wear [min]"] = no_duplicate_df["Tool wear [min]"].astype(float)

for i in temp_indexes:

    temp_series = no_duplicate_df.iloc[:,i]
    series_max = temp_series.max()
    series_min = temp_series.min()
    series_range = temp_series.max() - temp_series.min()

    for j in range(temp_series.shape[0]):

        no_duplicate_df.iat[j,i] = (no_duplicate_df.iat[j,i]-series_min)/series_range


no_dupe_normalized_df = no_duplicate_df

no_dupe_normalized_df.to_csv("PredicitiveMaintenanceDatasetPROCESSED.csv")